# Combine Data

Combine data sources into combined_data.pkl

In [11]:
import os
import pandas as pd

# 安全转换函数，避免数据错误导致读取失败
def safe_int(x):
    try:
        return int(x)
    except ValueError:
        return None

# 指定每列的数据类型，以优化读取速度
dtype = {
    'longitude': 'float32',  # 浮点型
    'latitude': 'float32',  # 浮点型
    'cog': 'float32',  # 浮点型
    'heading': 'float32',  # 浮点型
    'speed': 'float32',  # 浮点型
}

converters = {
    'mmsi': safe_int
}

# 获取当前脚本的路径
current_dir = os.getcwd()

# 指定主目录路径
source_data_path = os.path.join(current_dir, '..', 'data/raw')

# 输出文件夹路径
combined_data_path = os.path.join(current_dir, '..', 'output')

# 初始化一个空的DataFrame用于存放合并后的数据
df_combined = pd.DataFrame()

In [12]:
# 遍历所有文件夹和子文件夹
for root, dirs, files in os.walk(source_data_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)

            # 先读取文件头以检查包含的列
            temp_df = pd.read_csv(file_path, nrows=0)
            if 'post_time' in temp_df.columns:
                parse_dates = ['post_time']

                # 读取CSV文件
                df = pd.read_csv(file_path, dtype=dtype, converters=converters, parse_dates=parse_dates,
                                 low_memory=False)
                # 根据具体情况选择必要的字段进行提取
                df = df[['mmsi', 'longitude', 'latitude', 'cog', 'speed', 'post_time']]
            elif 'ts' in temp_df.columns:
                parse_dates = ['ts']

                # 读取CSV文件
                df = pd.read_csv(file_path, dtype=dtype, converters=converters, parse_dates=parse_dates,
                                 low_memory=False)
                # 根据具体情况选择必要的字段进行提取和重命名
                df = df[['mmsi', 'longitude', 'latitude', 'heading', 'speed', 'ts']]
                df.rename(columns={'ts': 'post_time', 'heading': 'cog'}, inplace=True)

            else:
                print(f"Skipping file {file} as it does not contain 'post_time' or 'ts' columns.")
                continue

            # 合并数据到df_combined中
            df_combined = pd.concat([df_combined, df], ignore_index=True)

            print(f"Processed {file_path}")

# # 将post_time列转换为日期时间格式，并移除多余位数
df_combined['post_time'] = df_combined['post_time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

Processed C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229.csv
Processed C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_2.csv
Processed C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_3.csv
Processed C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_4.csv
Processed C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_5.csv
Processed C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\data/raw\2. cq_tlx_dynamic_ais1 export\cq_tlx_dynamic_ais1_202405282248.csv
Processed C:\Projects\PycharmProjects

In [13]:
# 保存合并后的数据为Feather格式
df_combined.to_feather(combined_data_path + '/combined_data.feather')
print(f"Combined data saved to {combined_data_path + '/combined_data.feather'}")

# 保存合并后的数据为Pickel格式
df_combined.to_pickle(combined_data_path + '/combined_data.pkl')
print(f"Combined data saved to {combined_data_path + '/combined_data.pkl'}")

Combined data saved to C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\output/combined_data.feather
Combined data saved to C:\Projects\PycharmProjects\inland-ship-predict\scripts\..\output/combined_data.pkl


In [15]:
# 读取合并后的数据
df_combined = pd.read_pickle(combined_data_path + '/combined_data.pkl')
df_combined.head()

,mmsi,longitude,latitude,cog,speed,post_time
0,413769307.0,106.663643,29.571108,3600.0,0.0,2023-05-21 15:53:11.000000
1,413834492.0,106.668709,29.575460,0.0,0.0,2023-05-21 15:53:12.000000
2,413870065.0,106.631256,29.595894,0.0,0.0,2023-05-21 15:53:13.000000
3,413816773.0,106.664909,29.572378,0.0,0.0,2023-05-21 15:53:14.000000
4,413816773.0,106.664955,29.572483,177.0,1.0,2023-05-21 15:53:15.000000
